## Measuring model performance

Once we have created a embeddings catalogue from which the most "n" similiar images are returned we need to measure how this process is performing.<br>
This score will allow us to make improvements and meassure them, in order to find the best parameters and methods.

For the evaluation we will only use the customer photos, and each one of them will be compared with the retrieved photos. As a result we will obtain a rank startint from the most similar photo. Then we will define a K parámeter that will set the top k images from the returned rank.

For the evaluation we will use the following methods:
   - Accuracy@K: total number of positive cases / total number of cases
   - Recall@K: average of total number of positive cases for each "id" / total cases for each "id"

In [216]:
%matplotlib inline
import os
import sys
import re
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.patches as patches
import matplotlib.image as mpimg
import pandas as pd
import numpy as np
from tqdm import tqdm
from keras.applications.inception_v3 import InceptionV3 , preprocess_input
from keras.preprocessing import image
import cv2
from scipy.spatial.distance import cdist
# pip install h5py==2.8.0rc1 to disable the h5py warning

In [257]:
customer_df = pd.read_csv("./customer_df.csv")
retrieval_df = pd.read_csv("./retrieval_df.csv")

customer_df = customer_df[customer_df["category"] == "belts"]
retrieval_df = retrieval_df[retrieval_df["category"] == "belts"]

In [2]:
model = InceptionV3(weights="imagenet", include_top=False)

In [225]:
def sorted_aphanumeric(data):
    """This function sorts the data so that the dataframes, embedings and photos folder have the same order
    1.jpg, 2.jpg, ... instead of 1.jpg, 10.jpg, 2.jpg ...
    """
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [226]:
# Returns the array representation of all images in the path specified in raw format for display and processed format for the NN
def preprocess_img(dataset_path):
    img_paths = sorted_aphanumeric(os.listdir(dataset_path))
    img_paths = [os.path.join(dataset_path, img) for img in img_paths]
    raw_imgs = [image.load_img(img, target_size=(250,250)) for img in img_paths]
    proc_imgs = np.array([preprocess_input(np.expand_dims(image.img_to_array(img), axis=0)[0]) for img in raw_imgs])
    return raw_imgs, proc_imgs

#calculates the embeddings, write=True to get all the photos' embeddings to a csv, write=False for one-off customer queries
def embeddings(img_arrays):
    embeddings_img = [model.predict(np.expand_dims(img, axis=0)).flatten() for img in tqdm(img_arrays)]
    embeddings_img = np.array(embeddings_img, np.float16) #float16 reduces decimal precision but saves disk space  
    print("Writing to file")
    np.save("embeddings_belts_val.npy", embeddings_img) #npy files save/load faster than csv when working with ndarrays

def recommend_user(target_img, embs_catalogue, bbox=True, method="cosine"):
    if bbox == True:
        raw_img = crop_image(target_img) #crops to bbox and resizes
    else:
        raw_img = plt.imread(target_img) #allows testing with an image without bboxes
        raw_img = cv2.resize(raw_img, dsize=(250, 250), interpolation=cv2.INTER_CUBIC)

    proc_img = preprocess_input(np.expand_dims(raw_img, axis=0)) #preprocess to NN format
    embs_target = model.predict(proc_img).flatten() #extracts embedding
    
    distance = cdist(embs_catalogue, embs_target.reshape(1,-1), method) #run distances from user photo to catalogue
    rank = np.argsort(distance.ravel()) #ranks by similarity, returns the indices that would sort the distance array
    return rank

Now we need to create the catalogue for category we wish to evaluate

In [258]:
%%time
catalogue_path = "../photos_classified/belts/validation/retrieval/"
img_all_raw, img_all = preprocess_img(catalogue_path)

Wall time: 3.55 s


In [246]:
%%time
embeddings(img_all) #ONLY RUN THE FIRST TIME TO SAVE ALL EMBEDDINGS TO DISK

100%|██████████████████████████████████████████████████████████████████████████████| 8349/8349 [04:36<00:00, 30.19it/s]


Writing to file
Wall time: 4min 53s


In [261]:
del img_all # not needed anymore. deletes the img_all array to save memory

In [263]:
%%time 
#using np to load all embeddings, watch out, this will initially fill up all ram
embs_catalogue = np.load("./embeddings_belts_val.npy")

Wall time: 53.9 ms


In [264]:
#filters the customer data frame with the images from the validation folder only
maskCustom = pd.Series(
    [str(photo)+'.jpg' for photo in customer_df['photo']]).isin(os.listdir("../photos_classified/belts/validation/customer/"))
maskRetriev = pd.Series(
    [str(photo)+'.jpg' for photo in retrieval_df['photo']]).isin(os.listdir("../photos_classified/belts/validation/retrieval/"))

customer_val_df = customer_df.reset_index(drop=True)[maskCustom].sort_values(by='photo', ascending=True)
retrieval_val_df = retrieval_df.reset_index(drop=True)[maskRetriev].sort_values(by='photo', ascending=True)

In [284]:
def evaluate(customer_df, retrieval_df, customer_path, retrieval_path, embs_catalogue, k, method="cosine"):
    
    # Important: embedding has to have the same order as customer
    # path to all photos
    customer_photos = sorted_aphanumeric(os.listdir(customer_path))
    validation_photos = sorted_aphanumeric(os.listdir(retrieval_path))
    
    all_positives = 0                                    #all positive cases to calculate accuracy
    true_positives = np.empty(len(customer_photos))      #all true positives
    actual_positives = np.empty(true_positives.size)     #true positives + false negatives
    
    for n, target_img in enumerate(customer_photos):
        
        # read and crop imgs
        target_img = os.path.join(customer_path, target_img)
        img_array = mpimg.imread(target_img) #target_img is the path to the jpg
        photo_name = target_img.split("/")[-1].split(".")[0]
        image_customer = customer_df[customer_df["photo"] == int(photo_name)]
        if len(image_customer) > 1:
            image_customer["fix_bbox"] = image_customer["height"] + image_customer["width"] #aggs height and width, sorts them and picks the largest box
            image_customer = image_customer.sort_values(by="fix_bbox", ascending=False)[0:1] #this fixes a bug with some bboxes being 1px long

        x0 = int(image_customer["left"].values)
        y0 = int(image_customer["top"].values)
        width = int(image_customer["width"].values)
        height = int(image_customer["height"].values)

        image_cropped = img_array[y0:y0+height , x0:x0+width, :]  
        image_cropped = cv2.resize(image_cropped, dsize=(250, 250), interpolation=cv2.INTER_CUBIC) #cropped array

        # calculate similarity rank
        proc_img = preprocess_input(np.expand_dims(image_cropped, axis=0)) #keras preprocessing
        embs_target = model.predict(proc_img).flatten()
        distance = cdist(embs_catalogue, embs_target.reshape(1,-1), method) 
        rank = np.argsort(distance.ravel()) #index of retrieved photos in embeddings   
        rank = retrieval_df.iloc[rank]['id'][:k].tolist()

        image_id = image_customer['id'].tolist()[0]        
        actual_positives[n] = len(retrieval_df[retrieval_df["id"] == image_id])
        
#         print(n, rank[:10])
        
        if image_id in rank:
            all_positives += 1
            true_positives[n] = rank.count(image_id)
        else:
            true_positives[n] = 0
    
    accuracy = all_positives / len(customer_photos)
    total_recall = (true_positives/actual_positives).mean()
    
    print("Positive cases:", all_positives)
    print("\nAccuracy at {}: {:.2f}%".format(k, accuracy*100))
    print("Recall at {}: {:.2f}%\n".format(k, total_recall*100))

In [285]:
%%time
customer_validation_path = "../photos_classified/belts/validation/customer/"
retrieval_validation_path = "../photos_classified/belts/validation/retrieval/"
k = 20
evaluate(customer_val_df, retrieval_val_df, customer_validation_path, retrieval_validation_path, embs_catalogue, k)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Positive cases: 21

Accuracy at 20: 20.79%
Recall at 20: 6.68%

Wall time: 46.3 s
